# ESPN API ORM

Object Relational Mapper for ESPN API. 

POC:
- Scoreboard API

Additional:
- Odds
- Sport
- Team
- Season
- Boxscore
- Play by Play
- Player


## Simple

Game: 20230404_uconn_sandiegostate
- Info: Last years final
- Game_id: 401522202
- Home_id: 41
- Away_id: 21


In [162]:
import time

import requests
from enum import Enum

######################################
# Sport Consts
######################################

class ESPNSportTypes(Enum):
    COLLEGE_BASKETBALL = 'basketball/mens-college-basketball'


##################################
# Scoreboard Classes
##################################

from typing import List, Optional
from pydantic import BaseModel
import datetime

## Leagues

class Logo(BaseModel):
    href: str
    width: int
    height: int
    alt: str
    rel: List[str]
    lastUpdated: datetime.datetime

class League(BaseModel):
    id: str
    uid: str
    name: str
    abbreviation: str
    midsizeName: str
    slug: str
    season: 'Season'
    logos: List[Logo]
    calendarType: str
    calendarIsWhitelist: bool
    calendarStartDate: datetime.datetime
    calendarEndDate: datetime.datetime
    calendar: List[datetime.datetime]

    class Season(BaseModel):
        year: int
        startDate: datetime.datetime
        endDate: datetime.datetime
        displayName: str
        type: 'SeasonType'  

        class SeasonType(BaseModel):
            id: int
            type: int
            name: str
            abbreviation: str

## Day
class TeamRef(BaseModel):
    id: int

class VenueRef(BaseModel):
    id: int

class Day(BaseModel):
    date: str

class EventDate(BaseModel):
    date: datetime.datetime
    seasonType: int
    
## Events
    
class Link(BaseModel):
    rel: List[str]
    href: str
    text: Optional[str] = None
    isExternal: Optional[bool] = False
    isPremium: Optional[bool] = False

class Athlete(BaseModel):
    id: str
    fullName: str
    displayName: str
    shortName: str
    links: List['Link']
    headshot: Optional[str] = None
    jersey: str
    position: 'Position'
    team: 'TeamRef'
    active: bool

    class Position(BaseModel):
        abbreviation: str

class Address(BaseModel):
    city: str
    state: str

class CuratedRank(BaseModel):
    current: int

class Team(BaseModel):
    id: int
    uid: str
    location: str
    name: str
    abbreviation: str
    displayName: str
    shortDisplayName: str
    color: str
    alternateColor: Optional[str] = None
    isActive: bool
    venue: 'VenueRef'
    links: List['Link']
    logo: str
    conferenceId: str

class Competitor(BaseModel):
    id: str
    uid: str
    type: str
    order: int
    homeAway: str
    winner: Optional[bool] = None
    team: 'Team'
    score: str
    linescores: List['Linescore'] = []
    statistics: List['Statistic'] = []
    leaders: List['LeaderGroup'] = []
    curatedRank: Optional[CuratedRank] = None
    records: List['Record'] = []

    class Linescore(BaseModel):
        value: float

    class Statistic(BaseModel):
        name: str
        abbreviation: str
        displayValue: str

    class Record(BaseModel):
        name: str
        abbreviation: Optional[str] = None
        type: str
        summary: str

    class LeaderGroup(BaseModel):
        name: str
        displayName: str
        shortDisplayName: str
        abbreviation: str
        leaders: List['Leader'] = None

        class Leader(BaseModel):
            displayValue: str
            value: float
            athlete: 'Athlete'
            team: 'TeamRef'

class Competition(BaseModel):
    id: str
    uid: str
    date: datetime.datetime
    attendance: int
    type: 'Type'
    timeValid: bool
    neutralSite: bool
    conferenceCompetition: bool
    playByPlayAvailable: bool
    recent: bool
    venue: 'Venue'
    competitors: List[Competitor]

    class Venue(BaseModel):
        id: str
        fullName: str
        address: 'Address'
        capacity: int
        indoor: bool

    class Type(BaseModel):
        id: str
        abbreviation: str
    
class Event(BaseModel):
    id: str
    uid: str
    date: datetime.datetime
    name: str
    shortName: str
    season: 'Season'
    competitions: List[Competition]
    links: List['Link']
    status: 'Status'

    class Status(BaseModel):
        clock: float
        displayClock: str
        period: int
        type: 'Type'

        class Type(BaseModel):
            id: str
            name: str
            state: str
            completed: bool
            description: str
            detail: str
            shortDetail: str

    class Season(BaseModel):
        year: int
        type: int
        slug: str


class Scoreboard(BaseModel):
    leagues: List[League]
    groups: List[str] = None
    day: Optional[Day] = None
    eventsDate: Optional[EventDate] = None
    events: List[Event]


class ESPNBaseAPI:
    """
    ESPNBaseAPI class for making API requests to ESPN's sports data endpoints.

    Attributes:
        _base_url (str): The base URL for ESPN's public API.
        _core_url (str): The base URL for ESPN's core API.

    Methods:
        api_request(url: str, retry_count: int = 0) -> dict or None:
            Makes an API request to the specified URL.

            Args:
                url (str): The complete URL for the API request.
                retry_count (int): The number of times to retry the request in case of failure. Default is 0.

            Returns:
                dict or None: The JSON response from the API, or None if the request was unsuccessful.
                If the response indicates a 404 status code or an error, None is returned.

            Raises:
                Exception: Raises an exception if the request encounters an error after multiple retries.
                This is typically used when the request limit is exceeded (error code 2502).
    """

    def __init__(self):
        """
        Initializes an instance of the ESPNBaseAPI class.

        Attributes:
            _base_url (str): The base URL for ESPN's public API.
            _core_url (str): The base URL for ESPN's core API.
        """
        self._base_url = 'https://site.api.espn.com/apis/site/v2/sports'
        self._core_url = 'https://sports.core.api.espn.com/v2/sports'

    def api_request(self, url: str, retry_count: int = 0) -> dict or None:
        """
        Makes an API request to the specified URL.

        Args:
            url (str): The complete URL for the API request.
            retry_count (int): The number of times to retry the request in case of failure. Default is 0.

        Returns:
            dict or None: The JSON response from the API, or None if the request was unsuccessful.
            If the response indicates a 404 status code or an error, None is returned.

        Raises:
            Exception: Raises an exception if the request encounters an error after multiple retries.
            This is typically used when the request limit is exceeded (error code 2502).
        """
        try:
            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'
            }
            resp = requests.get(url=url, headers=headers)
            if resp.status_code == 404:
                return None
            res = resp.json()
            if 'error' in res:
                if res['error']['code'] == 404:  # No data
                    return None
            if 'code' in res:
                if res['code'] == 2502:
                    raise Exception('Flooded')  # Too many requests
                if res['code'] == 400:  # Data cant be found (wrong endpoint/wrong request)
                    return None
            return res
        except Exception as e:
            if retry_count >= 3:
                raise e
            time.sleep(5)
            print(f'URL error for {url}')
            self.api_request(url, retry_count=retry_count + 1)

    def get_scoreboard(self, sport: ESPNSportTypes, dates, limit=1000, groups=None) -> Scoreboard:
        """
        Retrieve scoreboard data for a specific sport.

        Args:
            sport (ESPNSportTypes): Type of sport.
            dates: Dates for events.
            limit (int): Limit of events to retrieve.
            groups: Groups for events.

        Returns:
            dict: API response containing scoreboard data.
        """
        url = f"{self._base_url}/{sport.value}/scoreboard?dates={dates}&limit={limit}"
        if groups is not None:
            url=f"{url}&groups={groups}"
        return Scoreboard(**self.api_request(url))




## Endpoints to Link



In [165]:
sport = ESPNSportTypes.COLLEGE_BASKETBALL
sport_str = sport.value
gameid = 401522202
home_id = 41
away_id = 21
espn_core_name = sport.value.split('/')[0] + '/leagues/' + sport.value.split('/')[1]
base_api = ESPNBaseAPI()

#res = base_api.api_request(f"{base_api._base_url}/{sport_str}/scoreboard?dates=20230403&limit=10")

In [166]:
res = base_api.get_scoreboard(sport, '2024')


In [167]:
res.events[0]

Event(id='401597194', uid='s:40~l:41~e:401597194', date=datetime.datetime(2024, 2, 21, 0, 0, tzinfo=TzInfo(UTC)), name='Tennessee Volunteers at Missouri Tigers', shortName='TENN @ MIZ', season=Season(year=2024, type=2, slug='regular-season'), competitions=[Competition(id='401597194', uid='s:40~l:41~e:401597194~c:401597194', date=datetime.datetime(2024, 2, 21, 0, 0, tzinfo=TzInfo(UTC)), attendance=0, type=Type(id='1', abbreviation='STD'), timeValid=True, neutralSite=False, conferenceCompetition=True, playByPlayAvailable=False, recent=False, venue=Venue(id='2071', fullName='Mizzou Arena', address=Address(city='Columbia', state='MO'), capacity=0, indoor=True), competitors=[Competitor(id='142', uid='s:40~l:41~t:142', type='team', order=0, homeAway='home', winner=None, team=Team(id=142, uid='s:40~l:41~t:142', location='Missouri', name='Tigers', abbreviation='MIZ', displayName='Missouri Tigers', shortDisplayName='Missouri', color='f1b82d', alternateColor='000000', isActive=True, venue=VenueR

In [142]:
a = Scoreboard(**res)
a.events[0].competitions[0].competitors[0].leaders[0].leaders

[Leader(displayValue='19', value=19.0, athlete=Athlete(id='4592965', fullName='Tristen Newton', displayName='Tristen Newton', shortName='T. Newton', links=[Link(rel=['playercard', 'desktop', 'athlete'], href='https://www.espn.com/mens-college-basketball/player/_/id/4592965/tristen-newton', text=None, isExternal=False, isPremium=False)], headshot='https://a.espncdn.com/i/headshots/mens-college-basketball/players/full/4592965.png', jersey='2', position=Position(abbreviation='G'), team=TeamRef(id=41), active=True), team=TeamRef(id=41))]